In [ ]:
# ============================================================================================
# PROBLEM B5
#
# Build and train a neural network model using the Daily Max Temperature.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from https://github.com/jbrownlee/Datasets
#
# Desired MAE < 0.2 on the normalized dataset.
# ============================================================================================

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:

def solution_B5():
    data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-max-temperatures.csv'
    urllib.request.urlretrieve(data_url, 'daily-max-temperatures.csv')

    time_step = []
    temps = []

    with open('daily-max-temperatures.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        step = 0
        for row in reader:
            temps.append(float(row[1])) # YOUR CODE HERE
            time_step.append(step) # YOUR CODE HERE
            step=step + 1

    series = np.array(temps)
    time = np.array(time_step)
    # Normalization Function. DO NOT CHANGE THIS CODE
    min=np.min(series)
    max=np.max(series)
    series -= min
    series /= max
    time=np.array(time_step)

    # DO NOT CHANGE THIS CODE
    split_time=2500

    time_train=time[:split_time]# YOUR CODE HERE
    x_train=series[:split_time]# YOUR CODE HERE
    time_valid=time[split_time:]# YOUR CODE HERE
    x_valid=series[split_time:]# YOUR CODE HERE

    # DO NOT CHANGE THIS CODE
    window_size=64
    batch_size=256
    shuffle_buffer_size=1000

    train_set=windowed_dataset(
        x_train, window_size, batch_size, shuffle_buffer_size)
    print(train_set)
    print(x_train.shape)

    model=tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=32, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        # YOUR CODE HERE.
        tf.keras.layers.Dense(1),
        tf.keras.layers.Lambda(lambda x : x * 400.0)
    ])
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(
        lambda epoch: 1e-8 * 10**(epoch / 20))
    optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer=optimizer,
                  metrics=["mae"])
    history = model.fit(train_set, epochs=50, callbacks=[lr_schedule])
    # YOUR CODE HERE
    return model

In [ ]:
# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_B5()
    model.save("model_B5.h5")

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
(2500,)
Epoch 1/50


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


10/10 [==============================] - 7s 317ms/step - loss: 0.3906 - mae: 0.7946 - lr: 1.0000e-08
Epoch 2/50
10/10 [==============================] - 3s 311ms/step - loss: 0.2201 - mae: 0.5366 - lr: 1.1220e-08
Epoch 3/50
10/10 [==============================] - 3s 313ms/step - loss: 0.1069 - mae: 0.3594 - lr: 1.2589e-08
Epoch 4/50
10/10 [==============================] - 3s 311ms/step - loss: 0.0823 - mae: 0.3375 - lr: 1.4125e-08
Epoch 5/50
10/10 [==============================] - 3s 312ms/step - loss: 0.0800 - mae: 0.3383 - lr: 1.5849e-08
Epoch 6/50
10/10 [==============================] - 3s 312ms/step - loss: 0.0749 - mae: 0.3256 - lr: 1.7783e-08
Epoch 7/50
10/10 [==============================] - 3s 313ms/step - loss: 0.0702 - mae: 0.3114 - lr: 1.9953e-08
Epoch 8/50
10/10 [==============================] - 3s 310ms/step - loss: 0.0669 - mae: 0.3015 - lr: 2.2387e-08
Epoch 9/50
10/10 [==============================] - 3s 313ms/step - loss: 0.0637 - mae: 0.2938 - lr: 2.5119e-08
Epo